## Libraries & settings

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 954 kB 1.9 MB/s 
     |████████████████████████████████| 138 kB 62.7 MB/s 
     |████████████████████████████████| 356 kB 52.0 MB/s 
     |████████████████████████████████| 3.5 MB 42.6 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [2]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd 
import os
import time
from lxml import html
from time import sleep
import re

# Scrapping ebay


In [ ]:
# https://www.youtube.com/watch?v=9ELd7XWD0PA&t=1067s

#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
#FireFoxDriverPath = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
#FireFoxProfile = webdriver.FirefoxProfile()
#FireFoxProfile.set_preference("general.useragent.override", user_agent)
url = "https://www.ebay.com/"
browser = wd
browser.implicitly_wait(7)
browser.get(url)

search_keyword = 'Playstation 5'
search_field = browser.find_element_by_id('gh-ac')
search_field.clear()
search_field.send_keys(search_keyword)
search_field.send_keys(Keys.ENTER)

total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
ebay_listings = []

current_page = 1
to_continue = True

In [ ]:
while to_continue:
    try:
        print('Processing page {0}'.format(current_page))

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        item_list = soup.find_all('li', 's-item--watch-at-corner')
        
        for listing in item_list:
            product_detail = {}

            product_detail['product title'] = listing.h3.text
            product_detail['product url'] = listing.a['href']
            
            listing_subtitles = listing.find_all('div', 's-item__subtitle')
            if len(listing_subtitles) == 2:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = listing_subtitles[1].text
            elif len(listing_subtitles) == 1:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = None
            else:
                listing_subtitle_1 = None
                listing_subtitle_2 = None

            product_detail['subtitle1'] = listing_subtitle_1
            product_detail['subtitle2'] = listing_subtitle_2


            # Star rating and product review count
            try:
                x_star_rating = listing.find('div', 'x-star-rating')
                stars = len(x_star_rating.find_all('svg', 'icon--star-full-small'))
                if int(stars) > 4: print(stars) 
                review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                
                product_detail['stars'] = stars
                product_detail['product review_count'] = review_count
            except Exception as e:
                product_detail['stars'] = ''
                product_detail['product review_count'] = ''

            product_detail['price'] = listing.find('span', 's-item__price').text

            try:
                product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
            except AttributeError:
                product_detail['shipping detail'] = 'Not Available'
                
            ebay_listings.append(product_detail)


        time.sleep(1)
        browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
        current_page +=1

        if current_page > 5:
            raise Exception('Stop')

    except NoSuchElementException:
        print('Last page {0}'.format(current_page))
        to_continue = False



In [ ]:
df = pd.DataFrame(ebay_listings)
df.head()

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail
0,Sony PlayStation 5 PS5 Console Disc Version *N...,https://www.ebay.com/itm/333896636458?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,049.00",Not Available
1,PS5 Sony PlayStation 5 Console Disc Version,https://www.ebay.com/itm/324483721021?epid=190...,Pre-Owned · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$859.99,Free shipping
2,Sony PlayStation 5 Console Disc Version (PS5) ...,https://www.ebay.com/itm/144223680923?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$949.99,Free shipping
3,Sony PlayStation 5 PS5 Console Disc Version Bl...,https://www.ebay.com/itm/193955663603?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$899.99,Free shipping
4,New ListingSony PlayStation 5 PS5 DISC Edition...,https://www.ebay.com/itm/203647656449?hash=ite...,Brand New · Sony PlayStation 5,None,,,$760.00,+$23.77 shipping


In [ ]:
df['amount'] = df.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

In [ ]:
df.sort_values(by='amount')

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail,amount
7,Turtle Beach Elite Atlas Pro Performance Heads...,https://www.ebay.com/itm/324391510282?hash=ite...,None,None,,,$59.95,Free shipping,5995
8,"Razer Kraken Gaming Headset For PC, PS4, PS5, ...",https://www.ebay.com/itm/144123396447?hash=ite...,None,None,,,$59.99,Free shipping,5999
9,SteelSeries Arctis 5 RGB Wired Gaming Headset ...,https://www.ebay.com/itm/334129396663?hash=ite...,None,None,,,$89.95,Free shipping,8995
191,New ListingSony PlayStation 5 PS5 Blu-Ray Edit...,https://www.ebay.com/itm/324827522681?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$102.50,+$77.61 shipping,10250
113,New Listing🎁Sony PlayStation 5 Console🎁Digital...,https://www.ebay.com/itm/265357244952?epid=250...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,461,$202.50,+$50.00 shipping,20250
...,...,...,...,...,...,...,...,...,...
129,New Listingplaystation 5 console,https://www.ebay.com/itm/304180922459?_trkparm...,Brand New · Sony PlayStation 5,None,,,"$1,124.00",+$10.00 shipping,112400
137,Sony PlayStation 5 Console PS5 [Disc Version] ...,https://www.ebay.com/itm/284478117917?epid=190...,Brand New · Sony PlayStation 5,None,0,1976,"$1,151.00",+$30.15 shipping,115100
64,Sony PlayStation 5 PS5 Console Disc Version BR...,https://www.ebay.com/itm/203581141972?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.95",Free shipping,119995
103,PS5 Sony Playstation 5 Console Disc Version 🔥B...,https://www.ebay.com/itm/304128005531?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.99",Free shipping,119999


In [ ]:
#df.to_excel('playstation 5 listings.xlsx', index=False)
print(df.iloc[103])

product title           PS5 Sony Playstation 5 Console Disc Version 🔥B...
product url             https://www.ebay.com/itm/304128005531?epid=190...
subtitle1               Brand New · Sony PlayStation 5 · Sony PlayStat...
subtitle2                                                            None
stars                                                                   0
product review_count                                                 1976
price                                                           $1,199.99
shipping detail                                             Free shipping
amount                                                             119999
Name: 103, dtype: object


In [ ]:
# https://towardsdatascience.com/scraping-multiple-amazon-stores-with-python-5eab811453a8

# scrapping Amazon



In [ ]:
# https://www.youtube.com/watch?v=_AeudsbKYG8

def get_url(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [ ]:
url = get_url('jetson nano')
print(url)

https://www.amazon.com.mx/s?k=jetson+nano&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1


In [ ]:
wd.get(url)

In [ ]:
# Extract the collection
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [ ]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [ ]:
len(results)

60

In [ ]:
# prototype the record
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com.mx' + atag.get('href')

In [ ]:
# price
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [ ]:
# stars
rating = item.i.text

In [ ]:
# find the reviews
review_count = item.find('span', {'class':'a-size-base'}).text

In [ ]:
# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result


In [ ]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
  record = extract_record(item)
  if record:
    records.append(record)

In [ ]:
print(records[0])

('NVIDIA Jetson Nano - Kit de desarrollador de 2 GB', '$1,482.99', '4.4 de 5 estrellas', '272', 'https://www.amazon.com.mx/NVIDIA-Jetson-Nano-Kit-desarrollador/dp/B08J157LHH/ref=sr_1_1?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=jetson+nano&qid=1634270630&sr=8-1')


In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page={}'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  for page in range(1,6):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  
  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [ ]:
amazon_scrapping_df = main('jetson nano')

In [ ]:
amazon_scrapping_df.head(10)

,Description,Price,Rating,Reviews,Url
0,NVIDIA Jetson Nano - Kit de desarrollador de 2 GB,"$1,482.99",4.4 de 5 estrellas,272,https://www.amazon.com.mx/NVIDIA-Jetson-Nano-K...
1,Módulo de cámara para NVIDIA Jetson Nano Devel...,"$1,019.00",,,https://www.amazon.com.mx/Development-pulgadas...
2,Waveshare Jetson Nano 2GB Developer Kit Packag...,"$3,449.99",4.6 de 5 estrellas,13,https://www.amazon.com.mx/Waveshare-Jetson-Nan...
3,TOP1 Intel AC8265 - NIC de red inalámbrica de ...,$433.32,,,https://www.amazon.com.mx/TOP1-Intel-AC8265-in...
4,Yahboom Carcasa de aleación de aluminio para N...,$585.03,4.0 de 5 estrellas,14,https://www.amazon.com.mx/Yahboom-aleaci%C3%B3...
5,Geekworm N100 - Carcasa de metal para Jetson N...,$499.00,4.3 de 5 estrellas,79,https://www.amazon.com.mx/Geekworm-NVIDIA-Encl...
6,Bicool Power Supply Adapter for Jetson Nano (A...,$317.46,,,https://www.amazon.com.mx/Adaptador-alimentaci...
7,"WENDi NVIDIA Jetson Nano Developer Kit, Small ...","$4,343.46",,,https://www.amazon.com.mx/WENDi-NVIDIA-Develop...
8,Geekworm UPS T208 18650 de 6 celdas (salida má...,"$1,252.93",3.1 de 5 estrellas,3,https://www.amazon.com.mx/Geekworm-tarjeta-exp...
9,Nvidia Jetson Xavier NX Developer Kit (8126740...,"$23,956.00",4.7 de 5 estrellas,120,https://www.amazon.com.mx/nVidia-Jetson-Xavier...


# Scrapping mercado libre

In [ ]:
# Search link
# https://listado.mercadolibre.com.mx/monitor#D[A:monitor]
# Page 2 

In [ ]:
def get_url(search_term):
  link = 'https://listado.mercadolibre.com.mx/{}'.format(search_term.replace(' ','-'))
  return link

In [ ]:
print(get_url('monitor curvo'))

https://listado.mercadolibre.com.mx/monitor-curvo


In [ ]:
url = get_url('monitor curvo')
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [ ]:
results = soup.find_all('li', {'class': 'ui-search-layout__item'})

In [ ]:
print(len(results))

50


In [ ]:
print(results[0])

<li class="ui-search-layout__item"><div class="ui-search-result__wrapper"><div class="andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default"><div class="ui-search-result__image"><a class="ui-search-link" href="https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&amp;position=1&amp;search_layout=stack&amp;type=product&amp;tracking_id=b02694ba-73e3-432e-80a7-5292855d8b8b" title='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V'><div class="carousel-container arrow-visible"><div class="slick-initialized slick-slider"><div class="slick-list" role="presentation"><div class="slick-track"><div class="slick-slide slick-active" data-index="0" style="outline:none" tabindex="-1"><img alt='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V' class="ui-search-result-image__element" he

In [ ]:
# prototype the record
item = results[0]
atag = item.h2.text.strip()
#description = atag.text.strip()
#url = 'https://listado.mercadolibre.com.mx/' + atag.get('href')

In [ ]:
print(item.a['href'])

https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&position=1&search_layout=stack&type=product&tracking_id=b02694ba-73e3-432e-80a7-5292855d8b8b


In [ ]:
price_parent = item.find('div', {'class':'ui-search-price__second-line'})
price = price_parent.find('span', 'price-tag-amount').text
print(price)

$6,599


In [ ]:
stars = item.find_all('svg', {'class':'ui-search-icon ui-search-icon--star ui-search-icon--star-full'})
print(len(stars))

5


In [ ]:
reviews = item.find('span', {'class':'ui-search-reviews__amount'}).text
print(reviews)

6


In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://listado.mercadolibre.com.mx/{}"
  search_term = search_term.replace(' ', '-')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '_Desde_{}_NoIndex_True'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  description = item.h2.text.strip()
  url = item.a['href']

  try:
    # price
    price_parent = item.find('div', {'class':'ui-search-price__second-line'})
    price = price_parent.find('span', 'price-tag-amount').text
  except AttributeError:
    return ''

  try:
    # reviews and rating
    stars = len(item.find_all('svg', {'class':'ui-search-icon ui-search-icon--star ui-search-icon--star-full'}))
    reviews = item.find('span', {'class':'ui-search-reviews__amount'}).text
  except AttributeError:
    stars = ''
    reviews = ''

  result = (description, price, stars, reviews, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  items_pp = 1
  to_continue = True
  
  while to_continue:

    try:
      wd.get(url.format(items_pp))
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      results = soup.find_all('li', {'class': 'ui-search-layout__item'})

      for item in results:
        record = extract_record(item)
      
        if record:
          records.append(record)
      items_pp += 50 
    
    except NoSuchElementException:
      print('Last page {0}'.format(current_page))
      to_continue = False

    if items_pp >= 151: to_continue = False 

  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [ ]:
mercado_libre_df = main('beyerdynamic')

In [ ]:
mercado_libre_df.head(10)

,Description,Price,Rating,Reviews,Url
0,Audífonos Beyerdynamic DT 990 PRO black y grey,"$4,040",4,4,https://www.mercadolibre.com.mx/audifonos-beye...
1,Almohadillas Akg Beyerdynamic Pioneer Universa...,$159,,,https://articulo.mercadolibre.com.mx/MLM-84341...
2,Beyerdynamic Audifonos Dt 990 Pro 250 Ohms,"$5,299",,,https://articulo.mercadolibre.com.mx/MLM-78029...
3,Almohadillas Akg Beyerdynamic Pioneer Terciope...,$199,,,https://articulo.mercadolibre.com.mx/MLM-90995...
4,Beyerdynamic Audifonos T1 (3rd Generation),"$29,999",,,https://articulo.mercadolibre.com.mx/MLM-94366...
5,Beyerdynamic Dt 990 Edition 600 Ohm Auriculare...,"$5,560",,,https://articulo.mercadolibre.com.mx/MLM-76993...
6,Beyerdynamic Audifonos Dt 240 Pro 34 Ohms,"$2,949",,,https://articulo.mercadolibre.com.mx/MLM-67097...
7,Beyerdynamic Dt-990 Pro 250ohms Buen Fin Navid...,"$4,500",,,https://articulo.mercadolibre.com.mx/MLM-83432...
8,Beyerdynamic Headphones In-ear Beat Byrd Black...,$809,,,https://articulo.mercadolibre.com.mx/MLM-78967...
9,Beyerdynamic Aventho Wireless Black Audifonos ...,"$10,980",,,https://articulo.mercadolibre.com.mx/MLM-76969...


# Scrapping wallmart mx

In [ ]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import time

In [ ]:


headers = {
    'Accept': 'application/json',
    'Referer': 'https://www.walmart.com.mx/productos?Ntt=xiaomi&page=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    'Content-Type': 'application/json',
    'Cookie': 'Cookie_1=value; TS014f2e15=01538efd7c027ce77e369e2bc6d77f0399abd465a588ddf5af91f7df97ebd11d0588faf61dd6c5ce95485742c700a31dd976d66ee4; akavpau_vp_walmart_ss=1634878568~id=8dbbff347a90a0bea31a88fa333da0d1'
  }

prods = pd.DataFrame([])

for page in range(1,5):

  url = f"https://www.walmart.com.mx/api/v2/page/search?Ntt=xiaomi&page={page}&size=48"
  r = requests.request("GET", url, headers=headers)
  data = json.loads(r.text)
  prods = prods.append(json_normalize(data['appendix']['SearchResults']['content']), ignore_index=True)
  time.sleep(4)
  print(f'Getting page {page} ...')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Getting page 1 ...
Getting page 2 ...
Getting page 3 ...
Getting page 4 ...


In [ ]:
print(len(prods))

192


In [ ]:
prods.head()


,id,productId,skuDisplayName,productSeoUrl,productRatings,weighable,freeShippingItem,skuPrice,bigItem,onDemand,bundle,preOrderable,pickUpEnabled,brandName,score,salesRank,variants,imageUrls.small,imageUrls.large,specialEventName,specialEventImage,unitOfMeasure,onSale
0,00750230095439,00750230095439,Smartphone Redmi Note 9 4G 4Gb 128Gb Negro Xia...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,5299.0,False,False,False,False,True,xiaomi,12.307381,0.000000e+00,[],/gm/3pp/asr/2fb44d2b-7e71-4272-9d84-9aba4f514d...,/gm/3pp/asr/2fb44d2b-7e71-4272-9d84-9aba4f514d...,NaN,NaN,NaN,NaN
1,00046990294261,00046990294261,Smartphone Xiaomi Redmi 9T 128Gb 4Gb CARBON GR...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,4950.0,False,False,False,False,True,xiaomi,12.307381,8.106390e-07,[],/gm/3pp/asr/bf20f6b8-66a3-4d6b-8dff-5f5e7d207a...,/gm/3pp/asr/bf20f6b8-66a3-4d6b-8dff-5f5e7d207a...,NaN,NaN,NaN,NaN
2,00693417772677,00693417772677,Xiaomi bocina Portable Bluetooth Speaker GRI...,/audio/bocinas-y-bafles/bocinas/xiaomi-bocina-...,0.0,False,No Free Shipping,579.0,False,False,False,False,True,xiaomi,12.307381,4.382517e-07,[],/gm/3pp/asr/36c1ec19-fa36-4e1b-9f44-aeeaa91fff...,/gm/3pp/asr/36c1ec19-fa36-4e1b-9f44-aeeaa91fff...,Envío en 48 horas,https://i5.walmartimages.com/dfw/fc2bdac6-b013...,NaN,NaN
3,00019099700077,00019099700077,Mi Truck Builder | Xiaomi | Robot Juguete de b...,/d/f/l/mi-truck-builder-xiaomi-robot-juguete-d...,0.0,False,No Free Shipping,499.0,False,False,False,False,True,xiaomi,12.307381,7.505917e-08,[],/gm/3pp/asr/33368528-14b9-453c-9c6c-c7183d7e5a...,/gm/3pp/asr/33368528-14b9-453c-9c6c-c7183d7e5a...,NaN,NaN,NaN,NaN
4,00890612903158,00890612903158,Xiaomi Redmi Note 10s 128 GB 6 GB RAM 64 MP Du...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,6198.0,False,False,False,False,True,xiaomi,12.307381,0.000000e+00,[],/gm/3pp/asr/42be58b2-68ca-4451-b304-6f1c24e90f...,/gm/3pp/asr/42be58b2-68ca-4451-b304-6f1c24e90f...,NaN,NaN,NaN,NaN


# scrapping sams

In [ ]:
import requests


'''
url = "https://www.sams.com.mx/sams/search?Ntt=pantalla&No=95&_=1635124678844"

payload={}
headers = {
  'authority': 'www.sams.com.mx',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'x-requested-with': 'XMLHttpRequest',
  'sec-gpc': '1',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty','user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
  'referer': 'https://www.sams.com.mx/search/Ntt=pantalla',
  'accept-language': 'es-419,es;q=0.9',
  'cookie': 'CLOUD=prod-dfw6; AskZipcode=Y; JSESSIONID=E6jpIxdvPnwb08EodlcMUKlo.restapp-267254238-3-1340057250; CL=prod-cdc6; TS01f4281b=0130aff232606065c9a93141954e73707a3607746e877ad7c4ddbc748f73833f6e066238f4ab9d7c1163b9ae2896f0b4ae403d3ce5; TS01e4bdf1=0130aff232606065c9a93141954e73707a3607746e877ad7c4ddbc748f73833f6e066238f4ab9d7c1163b9ae2896f0b4ae403d3ce5; akavpau_vp_sams=1635125317~id=52138295ab1e4d108b7d742a0a4d527b; akavpau_vp_sams=1635125629~id=e468083091fc27e21ab63294e4e659d0'
}

'''


#url = "https://www.sams.com.mx/sams/search?Ntt=iphone&_=1635024068179"
url = "https://www.sams.com.mx/sams/search?Ntt=pantalla&No=95"

payload={}
headers = {
  'authority': 'www.sams.com.mx',
  'accept': 'application/json, text/javascript, */*; q=0.01',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'x-requested-with': 'XMLHttpRequest',
  'sec-gpc': '1',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': 'https://www.sams.com.mx/search/Ntt=pantalla',
  'accept-language': 'es-419,es;q=0.9',
  'cookie': 'CLOUD=prod-dfw6; JSESSIONID=h2MCuJH-5QoJQlr1FqYDmMVo.restapp-267254238-3-1340057250; AskZipcode=Y; CL=prod-cdc03; TS01f4281b=0130aff232911a4d804ff0629a31fa7bdfb057a225b7a0751bc1b5870768fd2ecc4571c25bd7e0702a120c39026ec0f9ab40a7f822; TS01e4bdf1=0130aff232911a4d804ff0629a31fa7bdfb057a225b7a0751bc1b5870768fd2ecc4571c25bd7e0702a120c39026ec0f9ab40a7f822; akavpau_vp_sams=1635024668~id=a7e5df36acebdff0445b69a9d1703bf3; akavpau_vp_sams=1635024947~id=c1b3151c319a92779e866250aacb2bce'
}

response = requests.request("GET", url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')


In [ ]:
print(soup.text)

{"leftArea":[{"name":"GuidedNav","contents":[{"name":"Navigation Container","navigation":[{"displayName":"Categoría","dimensionName":"product.category","refinements":[{"navigationState":"/meses-sin-intereses/_/N-8py?Ntt=pantalla","count":132,"label":"Meses sin Intereses","dimensionName":null,"multiselect":false,"properties":{"category.repositoryId":"cat230005","DisplayOrder":"0","category.catalogs.repositoryId":"catalog20001","category.rootCatalogId":"catalog20001","category.imgurl":"/images/banner-images/escaparates/escaparate-msi-170418.png","category.siteId":"100001","category.mobileThumbnailImage":"/images/App/Banners_app/icono-msi-app-1-191016.jpg","SourceId":"cat230005","category.ancestorCatalogIds":"catalog20001"}},{"navigationState":"/electronica-y-computacion/_/N-84u?Ntt=pantalla","count":128,"label":"Electrónica y Computación","dimensionName":null,"multiselect":false,"properties":{"category.repositoryId":"cat30186","DisplayOrder":"0","category.catalogs.repositoryId":"catalog2

In [ ]:
#['contents']['records']
print(json.loads(soup.text)['mainArea'][10]['contents'][0]['records'])

[{'records': [{'records': None, 'numRecords': 0, 'attributes': {'sku.lastPrice': ['12888.78'], 'maxQuantity': ['60.0'], 'HomeDelivery': ['FX'], 'sku.finalPrice': ['11149.68'], 'skuStatusFlag': ['A'], 'sku.repositoryId': ['980029842'], 'sku.weighable': ['0'], 'product.seoURL': ['[/pantallas/pantalla-tcl-50-pulgadas-uhd-4k-android-tv-50a445/980029842]'], 'product.HomeDelivery': ['FX'], 'expressDelivery': ['Express Delivery'], 'record.spec': ['sku-980029842..0980029842.catalog20001.es__MX'], 'priceStrikeOff': [1], 'PickAtStore': ['Pick at Store'], 'STR0000009999': ['SELLABLE'], 'sku.isBigItem': ['0'], 'skuDisplayName': ['Pantalla TCL 50 Pulgadas UHD 4K Android TV 50A445']}}], 'numRecords': 1, 'attributes': {'sku.availableStores': ['0000004727', '0000004728', '0000004746', '0000004790', '0000004791', '0000004792', '0000004801', '0000004805', '0000004827', '0000004832', '0000004841', '0000004862', '0000004877', '0000004878', '0000004879', '0000004901', '0000004910', '0000004911', '000000491

In [ ]:
df = json_normalize(json.loads(soup.text)['mainArea'][10]['contents'][0]['records'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [ ]:
df.head()

,records,numRecords,attributes.sku.availableStores,attributes.sku.lastPrice,attributes.product.smallImage.url,attributes.HomeDelivery,attributes.product.family,attributes.product.displayName,attributes.expressDeliveryEligible,attributes.sku.finalPrice,attributes.PickupInStock,attributes.sku.repositoryId,attributes.sku.weighable,attributes.sku.oosStores,attributes.product.seoURL,attributes.sku.smallImage.url,attributes.expressDelivery,attributes.PickupEligible,attributes.sku.displayName,attributes.DeliveryInStock,attributes.sku.unit_of_measure,attributes.DeliveryEligible,attributes.record.type.raw,attributes.product.department,attributes.record.id,attributes.expressDeliveryInStock,attributes.Brand,attributes.product.repositoryId,attributes.PickAtStore,attributes.PaymentPlan,attributes.product.displayText,attributes.skuDisplayName,attributes.product.fineline,attributes.sku.preOrderEndDate,attributes.No_Disponible_and_Remind_Me
0,"[{'records': None, 'numRecords': 0, 'attribute...",1,"[0000004727, 0000004728, 0000004746, 000000479...",[12888.78],[/images/product-images/img_small/980029842s.jpg],[FX],"[12 Meses sin Intereses, 3, 6 ó 9 Meses sin In...",[Pantalla],[false],[11149.68],[true],[980029842],[0],[0000006039],[[/pantallas/pantalla-tcl-50-pulgadas-uhd-4k-a...,[/images/product-images/img_small/980029842s.jpg],[Express Delivery],[true],[Pantalla],[true],[eaches],[true],"[product, sku]","[Electrónica y Computación, Meses sin Interese...",[sku-980029842..0980029842.catalog20001.es__MX],[false],[TCL],[0980029842],[Pick at Store],"[13, 3, 6, 9]",[Pantalla TCL 50 Pulgadas UHD 4K Android TV 50...,[Pantalla TCL 50 Pulgadas UHD 4K Android TV 50...,"[Pantallas, Tecnologia y Electrónica]",NaN,NaN
1,"[{'records': None, 'numRecords': 0, 'attribute...",1,"[0000004728, 0000004746, 0000004790, 000000479...",[14832.48],[/images/product-images/img_small/980029844s.jpg],[FX],"[12 Meses sin Intereses, 3, 6 ó 9 Meses sin In...",[Pantalla],[false],[13195.68],[true],[980029844],[0],[0000006039],[[/pantallas/pantalla-tcl-55-pulgadas-uhd-4k-a...,[/images/product-images/img_small/980029844s.jpg],[Express Delivery],[true],[Pantalla],[true],[eaches],[true],"[product, sku]","[Categorías_Exclusivas_App, Cuponera Sams, Ele...",[sku-980029844..0980029844.catalog20001.es__MX],[false],[TCL],[0980029844],[Pick at Store],"[13, 3, 6, 9]",[Pantalla TCL 55 pulgadas UHD 4K Android TV 55...,[Pantalla TCL 55 pulgadas UHD 4K Android TV 55...,"[Pantallas, Tecnologia y Electrónica]",NaN,NaN
2,"[{'records': None, 'numRecords': 0, 'attribute...",1,"[0000004727, 0000004728, 0000004746, 000000479...",[3784.07],[/images/product-images/img_small/980032078s.jpg],[FX],"[Electrónica, Pantallas y Audio, Promociones B...",[Pantalla],[false],[3784.07],[true],[980032078],[0],"[0000006039, 0000006236, 0000006309]",[[/pantallas/pantalla-sansui-24-pulgadas-hd-sm...,[/images/product-images/img_small/980032078s.jpg],[Express Delivery],[true],[Pantalla],[true],[eaches],[true],"[product, sku]","[Electrónica y Computación, Meses sin Interese...",[sku-980032078..0980032078.catalog20001.es__MX],[false],[Sansui],[0980032078],[Pick at Store],"[13, 6, 9]",[Pantalla Sansui 24 Pulgadas HD Smart TV],[Pantalla Sansui 24 Pulgadas HD Smart TV],"[Pantallas, Tecnologia y Electrónica]",NaN,NaN
3,"[{'records': None, 'numRecords': 0, 'attribute...",1,"[0000004727, 0000004728, 0000004746, 000000479...",[18003.77],[/images/product-images/img_small/980029446s.jpg],[FX],"[12 Meses sin Intereses, 3, 6 ó 9 Meses sin In...",[Pantalla],[false],[14934.78],[true],[980029446],[0],[0000006039],[[/pantallas/pantalla-lg-55-pulgadas-smart-tv-...,[/images/product-images/img_small/980029446s.jpg],[Express Delivery],[true],[Pantalla],[true],[eaches],[true],"[product, sku]","[Electrónica y Computación, LG, Meses sin Inte...",[sku-980029446..0980029446.catalog20001.es__MX],[false],[LG],[0980029446],[Pick at Store],"[13, 3, 6, 9]",[Pantalla LG 55 Pulgadas Smart TV UHD 4K AI Th...,[Pantalla LG 55 Pulgadas Smart TV UHD 4K AI Th..

In [ ]:
df.shape

(32, 35)

# scraping Ali express

In [ ]:
header = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [ ]:
html = requests.get('https://es.aliexpress.com/premium/iphone.html?trafficChannel=ppc&d=y&CatId=0&SearchText=iphone&ltype=premium&SortType=default&page=2', headers = header)

In [ ]:
html.status_code

200

In [ ]:
cookies = html.cookies
print(cookies)

<RequestsCookieJar[<Cookie aep_usuc_f=site=esp&c_tp=EUR&region=US&b_locale=es_ES for .aliexpress.com/>, <Cookie intl_common_forever=DUFJBqsVrzgru0QpRIP1FNF2HQKIKNXWOBiBRks/ofa1BGcsBYmb5Q== for .aliexpress.com/>, <Cookie intl_locale=es_ES for .aliexpress.com/>, <Cookie xman_us_f=x_locale=es_ES&x_l=0&x_c_chg=1&x_as_i=%7B%22aeuCID%22%3A%22%22%2C%22cookieCacheEffectTime%22%3A1635223034045%2C%22isCookieCache%22%3A%22Y%22%2C%22ms%22%3A%220%22%7D&acs_rt=b53f16fad41b4ca5a90869abce10461a for .aliexpress.com/>, <Cookie JSESSIONID=BDE98D12547A6EAFA5E651237C411175 for es.aliexpress.com/>]>


In [ ]:
html = requests.get('https://es.aliexpress.com/premium/iphone.html?trafficChannel=ppc&d=y&CatId=0&SearchText=iphone&ltype=premium&SortType=default&page=2', headers = header, cookies = cookies)

In [ ]:
# this does not retrive the whole info
bsobj = soup(html.content)

In [ ]:
asession = AsyncHTMLSession()

r = asession.get('https://es.aliexpress.com/premium/iphone.html?trafficChannel=ppc&d=y&CatId=0&SearchText=iphone&ltype=premium&SortType=default&page=2')

In [ ]:
r.html.arender()

AttributeError: ignored

In [ ]:
header = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
cookies = '<RequestsCookieJar[<Cookie aep_usuc_f=site=esp&c_tp=EUR&region=US&b_locale=es_ES for .aliexpress.com/>, <Cookie intl_common_forever=DUFJBqsVrzgru0QpRIP1FNF2HQKIKNXWOBiBRks/ofa1BGcsBYmb5Q== for .aliexpress.com/>, <Cookie intl_locale=es_ES for .aliexpress.com/>, <Cookie xman_us_f=x_locale=es_ES&x_l=0&x_c_chg=1&x_as_i=%7B%22aeuCID%22%3A%22%22%2C%22cookieCacheEffectTime%22%3A1635223034045%2C%22isCookieCache%22%3A%22Y%22%2C%22ms%22%3A%220%22%7D&acs_rt=b53f16fad41b4ca5a90869abce10461a for .aliexpress.com/>, <Cookie JSESSIONID=BDE98D12547A6EAFA5E651237C411175 for es.aliexpress.com/>]>'

In [ ]:
wd.get('https://es.aliexpress.com/premium/iphone.html?trafficChannel=ppc&d=y&CatId=0&SearchText=iphone&ltype=premium&SortType=default&page=1')
sleep(1)

In [ ]:
header = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
cookies = '<RequestsCookieJar[<Cookie aep_usuc_f=site=esp&c_tp=EUR&region=US&b_locale=es_ES for .aliexpress.com/>, <Cookie intl_common_forever=DUFJBqsVrzgru0QpRIP1FNF2HQKIKNXWOBiBRks/ofa1BGcsBYmb5Q== for .aliexpress.com/>, <Cookie intl_locale=es_ES for .aliexpress.com/>, <Cookie xman_us_f=x_locale=es_ES&x_l=0&x_c_chg=1&x_as_i=%7B%22aeuCID%22%3A%22%22%2C%22cookieCacheEffectTime%22%3A1635223034045%2C%22isCookieCache%22%3A%22Y%22%2C%22ms%22%3A%220%22%7D&acs_rt=b53f16fad41b4ca5a90869abce10461a for .aliexpress.com/>, <Cookie JSESSIONID=BDE98D12547A6EAFA5E651237C411175 for es.aliexpress.com/>]>'
myUrl = 'https://es.aliexpress.com/wholesale?catId=0&initiative_id=SB_20211030184223&SearchText=iphone+11'

In [ ]:
#Ali express data frame
df_aliexpress = pd.DataFrame(columns = ['description', 'price', 'link', 'sold', 'store','shipping'])


for page_nb in range(1, 3):

    wd.get(myUrl + '&ltype=wholesale&SortType=default&page={}'.format(page_nb))

    # scroll down page slowly to wait for hidden products to show
    start_time = time.time()
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    seconds = 5

    while True:
        current_time = time.time()
        elapsed_time = current_time - start_time

        wd.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

        if elapsed_time > seconds:
            break

    # get html content
    tree = html.fromstring(wd.page_source)
    
    title = tree.xpath('//div[@class="JIIxO"]//h1[@class="_18_85"]/text()')
    price = re.findall('[^€]+€', ''.join(tree.xpath('//div[@class="JIIxO"]//div[@class="mGXnE _37W_B"]//span/text()')))
    link = [('https://es.aliexpress.com' + i) for i in tree.xpath('//div[@class="JIIxO"]//a[@class="_2f4Ho _3t7zg"]/@href')]
    sold = tree.xpath('//div[@class="JIIxO"]//span[@class="_1kNf9"]/text()')
    store = [('https:' + i) for i in tree.xpath('//div[@class="JIIxO"]//a[@role="store"]/@href')]
    shipping = tree.xpath('//div[@class="JIIxO"]//span[@class="_2jcMA"]/text()')

    products = list(zip(title, price, link, sold, store, shipping))

    df_aliexpress = df_aliexpress.append(products, ignore_index = True)

    #print(len(tree.xpath('//div[@class="JIIxO"]//h1[@class="_18_85"]')))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [ ]:
df_aliexpress

,0,1,2,3,4,5,description,link,price,shipping,sold,store
0,"Apple-teléfono móvil iPhone 11 Original, smart...",428.53€,https://es.aliexpress.com/item/100500332370564...,74 vendido(s),https://es.aliexpress.com/store/5324008,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple-teléfono inteligente iPhone 11 Original ...,448.87€,https://es.aliexpress.com/item/100500151912939...,43 vendido(s),https://es.aliexpress.com/store/1110648,"+ envío: € 92,28",NaN,NaN,NaN,NaN,NaN,NaN
2,"Apple-teléfono inteligente iPhone 11 Pro, Orig...",575.82€,https://es.aliexpress.com/item/100500245355153...,75 vendido(s),https://es.aliexpress.com/store/5324008,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
3,"Apple-Smartphone iPhone 11 pro desbloqueado, t...",575.44€,https://es.aliexpress.com/item/100500266456325...,87 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
4,Apple-teléfono inteligente iPhone 11 desbloque...,437.47€,https://es.aliexpress.com/item/100500340290627...,18 vendido(s),https://es.aliexpress.com/store/1110648,"+ envío: € 92,28",NaN,NaN,NaN,NaN,NaN,NaN
5,"Apple-teléfono inteligente iPhone 11 pro max, ...",666.15€,https://es.aliexpress.com/item/100500305711728...,53 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
6,"Apple-teléfono inteligente iPhone 11 4G LTE, O...",417.31€,https://es.aliexpress.com/item/100500294000979...,25 vendido(s),https://es.aliexpress.com/store/709663,"+ envío: € 46,14",NaN,NaN,NaN,NaN,NaN,NaN
7,"Apple-teléfono inteligente iPhone 11, Original...",459.72€,https://es.aliexpress.com/item/100500261641230...,69 vendido(s),https://es.aliexpress.com/store/911615234,"+ envío: € 13,84",NaN,NaN,NaN,NaN,NaN,NaN
8,"Apple-teléfono inteligente iPhone 11 Pro Max, ...",666.11€,https://es.aliexpress.com/item/100500319950705...,52 vendido(s),https://es.aliexpress.com/store/911614417,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN
9,Apple-teléfono inteligente iPhone 11 desbloque...,482.18€,https://es.aliexpress.com/item/100500263723176...,46 vendido(s),https://es.aliexpress.com/store/1053035,Envío gratis,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#print((''.join(tree.xpath('//div[@class="JIIxO"]//div[@class="mGXnE _37W_B"]//span/text()'))).split('€'))
print(re.findall('[^€]+€', ''.join(tree.xpath('//div[@class="JIIxO"]//div[@class="mGXnE _37W_B"]//span/text()'))))

['428.53€', '448.87€', '575.44€', '437.47€', '666.15€', '459.72€', '482.18€', '592.65€', '553.72€', '575.54€', '1.39€', '1.22€', '7.53€']


In [ ]:
print(tree.xpath('//div[@class="JIIxO"]//span[@class="_1kNf9"]/text()'))

['74 vendido(s)', '43 vendido(s)', '87 vendido(s)', '18 vendido(s)', '53 vendido(s)', '69 vendido(s)', '46 vendido(s)', '30 vendido(s)', '1 vendido(s)', '13 vendido(s)', '27803 vendido(s)', '31474 vendido(s)', '48 vendido(s)', '8357 vendido(s)']


In [ ]:
print(tree.xpath('//div[@class="JIIxO"]//span[@class="_2jcMA"]/text()'))

['Envío gratis', 'Envío gratis', '+ envío: € 92,28', '+ envío: € 13,84', '+ envío: € 13,84', 'Envío gratis', '+ envío: € 92,28', '+ envío: € 13,84', '+ envío: € 46,14', '+ envío: € 46,14', 'Envío gratis', 'Envío gratis', 'Envío gratis', 'Envío gratis']


In [ ]:
'''
https://www.amazon.com.mx/Crawler-Control-Electric-Waterproof-Included/dp/B087NXVYF2/ref=pd_day0_2/146-0449573-7191119?pd_rd_w=5jHPH&pf_rd_p=1c8f9346-a87e-48c7-994a-82c93e96af91&pf_rd_r=76AYTQ5FQTV6VCEBHE6A&pd_rd_r=3fdccf3c-3e6a-44ad-b4a6-b96105683dec&pd_rd_wg=HNu3a&pd_rd_i=B087NXVYF2&psc=1
https://github.com/NVIDIA-AI-IOT/jetracer#cars
https://gsurma.medium.com/jetson-self-driving-toy-car-part-1-4341c139c0f2
https://es.aliexpress.com/w/wholesale-jetson-nano-donkey-car.html
https://www.amazon.com/-/es/Yahboom-codificaci%C3%B3n-autom%C3%A1tico-seguimiento-reconocimiento/dp/B07ZYJYGZ5/ref=sr_1_29?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=16RZNYMCX3SK&dchild=1&keywords=donkey+car&qid=1635478468&sprefix=donkey+ca%2Caps%2C294&sr=8-29
https://www.amazon.com.mx/s?k=JetRacer+AI+Kit&ref=nb_sb_noss

'''

# scrapping copel

In [ ]:
#url = 'https://www.coppel.com/SearchDisplay?categoryId=&storeId=10151&catalogId=10051&langId=-5&sType=SimpleSearch&resultCatEntryType=2&showResultsPage=true&searchSource=Q&pageView=&pageGroup=Search&beginIndex=0&pageSize=12&searchTerm=iphone+13&authToken=-1002%252CJdh5JVzqovGDGuZIS9A0yAc11LCiDPi1CWNOoTX%252B9xo%253D'
#url = 'https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=12&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=12&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151'
#url = 'https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=24&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=12&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151'

In [ ]:
# Example page 1 index 01
url = 'https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=01&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151'

In [9]:
def get_url(search_term):
  """Genetare the Url from search term"""
  template = "https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=i{}&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=01&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [10]:
url = get_url('iphone 13')
print(url)

https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iiphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=01&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151


In [11]:
wd.get(url)
# Extract the collection
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [12]:
results = soup.find_all('div', {'class': 'product'})
print(len(results))

13


In [25]:
# title
item = results[2]
atag = item.find('div', {'class':'product_name'}).find('a').text  
print(atag)


Telcel Apple 13 mini 128 GB (PRODUCT)RED



In [ ]:
# price
price = item.find('span',{'class':'unique_price'})
print(price.text)


				$19,049
			


In [ ]:
# link
link = item.find('div', {'class':'product_name'}).find('a').get('href')
print(link)

https://www.coppel.com/telcel-apple-iphone-13-mini-128-gb-pink-pm-2518783


In [22]:
# script completed for coppel

def get_url(search, number):
  """Genetare the Url from search term"""
  
  number = str(number)
  if str(number) == '0': number = '01' 
  search = search.replace(' ', '+')
  template = f"https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=i{search}&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex={number}&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151"
  print(template)
  return template

  
def extract_record(item):
  """scrapping each item"""

  # description and url
  print(item)
  description = item.find('div', {'class':'product_name'}).find('a').text  
  url = item.find('div', {'class':'product_name'}).find('a').get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return 'Not found'

  result = (description, price, url)

  return result


def main(search):
  """run main program"""
  # set webdrriver if needed

  number = 0
  keep_going = True
  records = []
  #url = get_urls(search_term)

  while keep_going:

    if number == 48: keep_going = False
    wd.get(get_url(search, number))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'class': 'product'})
    print(len(results))

    for item in results:
      record = extract_record(item)
    if record:
      records.append(record)

    number += 12

  wd.close()
    
  return pd.DataFrame(records, columns = ['Description', 'Price', 'Url'])

In [23]:
df_coppel = main('iphone 13')

https://www.coppel.com/SearchDisplay?sType=SimpleSearch&ddkey=ProductListingView_6_-3011_1208&fromPage=catalogEntryList&searchTerm=iiphone+13&urlRequestType=Base&catalogId=10051&resultType=products&disableProductCompare=false&showResultsPage=true&beginIndex=01&resultCatEntryType=2&langId=-5&enableSKUListView=false&pageSize=1&ajaxStoreImageDir=%2Fwcsstore%2FCoppelStoreFrontAssetStore%2F&storeId=10151
13


KeyError: ignored

In [ ]:
df_coppel.head()

In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page={}'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return 'Not found'

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  for page in range(1,6):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  
  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

# scrapping elektra